<div class="alert alert-block alert-success">

**Reference Guide for R (student resource) -** Check out our <a href = "https://docs.google.com/document/d/1dXT3NO4QjtbIrq29fH6pXHKgt7odZN7BIeU_TBoia4I/edit?usp=sharing">reference guide</a> for a full listing of useful R commands for this project. 

<div>

<img src="https://skewthescript.org/s/money_college_scale.jpg">

## Data Science Project: Use data to determine the best and worst colleges for conquering student debt.

### Notebook 4: Machine Learning

Does college pay off? We'll use some of the latest data from the US Department of Education's <a href="https://collegescorecard.ed.gov/data/">College Scorecard Database</a> to answer that question. 

In this notebook (the 4th of 4 total notebooks), you'll use R to add polynomial terms to your multiple regression models (i.e. polynomial regression). Then, you'll use the principles of machine learning to tune models for a prediction task on *unseen* data.

In [ ]:
## Run this code but do not edit it. Hit Ctrl+Enter to run the code.
# This command downloads a useful package of R commands
library(coursekata)

<div class="alert alert-block alert-success">

### The Dataset (`four_year_colleges.csv`)

**General description** - In this notebook, we'll be using the `four_year_colleges.csv` file, which only includes schools that offer four-year bachelors degrees and/or higher graduate degrees. Community colleges and trade schools often have different goals (e.g. facilitating transfers, direct career education) than institutions that offer four-year bachelors degrees. By comparing four-year colleges only to other four-year colleges, we'll have clearer analyses and conclusions. 

This data is a subset of the US Department of Education's <a href="https://collegescorecard.ed.gov/data/">College Scorecard Database</a>. The data is current as of the 2020-2021 school year.

**Description of all variables:** See <a href="https://docs.google.com/document/d/1C3eR6jZQ2HNbB5QkHaPsBfOcROZRcZ0FtzZZiyyS9sQ/edit">here</a>

**Detailed data file description:** See <a href="https://docs.google.com/spreadsheets/d/1fa_Bd3_eYEmxvKPcu3hK2Dgazdk-9bkeJwONMS6u43Q/edit?usp=sharing">here</a></div>

In [ ]:
## Run this code but do not edit it. Hit Ctrl+Enter to run the code.
# This command downloads data from the file 'colleges.csv' and stores it in an object called `dat`
dat <- read.csv('https://skewthescript.org/s/four_year_colleges.csv')

### 1.0 - Motivating non-linear regression

So far, we've focused entirely on **linear regression** and **multiple linear regression** models, which use linear functions to relate predictors (e.g. `net_tuition`,`grad_rate`,`pct_PELL`) to the outcome (`default_rate`). 

In this notebook, we're going to investigate ways to model **non-linear** relationships. To make this task a bit more manageable at the start, let's reduce the size of our dataset by taking a random sample of 20 colleges from the `dat` dataframe. We will store our sample in a new R dataframe called `sample_dat`.

In [ ]:
## Run this code but do not edit it
# create a dataset to train the model with 20 randomly selected observations
set.seed(2)
sample_dat <- sample(dat, size = 20)

**Note:** When getting a random sample, we'll get different results each time we run our code because it's ... well ... random. This can be quite annoying. So, in the code above, we used the command `set.seed(2)`. This ensures that each time the code is executed, we get the same results for our random sample - the results stored in seed `2`. We could have also set the seed to `1` or `3` or `845` or `12345`. The seed numbers serve merely as a unique ID that corresponds to a certain result from a random draw. By setting a certain seed, we'll always get a certain random draw.

<div class="alert alert-block alert-warning">

**1.1** Let's take a look at our sample data set. Print out the `head` and `dim` of `sample_dat`.
    
</div>

In [ ]:
# Your code goes here


In [ ]:
# Your code goes here


<div class="alert alert-block alert-info">

**Check yourself:** The dimensions of `sample_dat` should be 20 rows and 27 columns.</div>

In prior notebooks, we focused on institutional and economic predictors of student loan default rates. In this notebook, we'll begin by analyzing an *academic* variable: `SAT_avg`. This variable shows the average SAT score of students who matriculate to a college.

The following code creates a scatterplot of the relationship between `SAT_avg` (predictor) and `default_rate` (outcome) from the dataset `sample_dat`:

In [ ]:
## Run this code but do not edit it
# create scatterplot: default_rate ~ SAT_avg
gf_point(default_rate ~ SAT_avg, data = sample_dat)

<div class="alert alert-block alert-warning">

**1.2 -** Describe the direction of the relationship between `SAT_avg` and `default_rate`. Is it positive or negative? Why do you think this is?
    
</div>

**Double-click this cell to type your answer here:** 

<div class="alert alert-block alert-warning">

**1.3 -** Create the same scatterplot as above, but with the simple linear model between `default_rate` (outcome) and `SAT_avg` (predictor) overlayed on top.

**Hint:** Recall the `gf_lm` command from notebook 2.
    
</div>

In [ ]:
# Your code goes here


<div class="alert alert-block alert-warning">

**1.4 -** Would you say that this model provides a "good" fit for this dataset? Explain.
    
</div>

**Double-click this cell to type your answer here:** 

<div class="alert alert-block alert-warning">

**1.5 -** Use the `lm` command to fit the linear regression model, where we use `SAT_avg` (predictor) to predict `default_rate` (outcome) in the dataset `sample_dat`. Store the model in a variable named `sat_model_1` and use the `summary` command to print out information about the model fit.
    
</div>

In [ ]:
# Your code goes here


<div class="alert alert-block alert-info">

**Check yourself:** The $R^2$ value shown in the model summary should be 0.4571 </div>

<div class="alert alert-block alert-warning">

**1.6** - Does the model's $R^2$ value indicate that this model provides a strong fit for this dataset? Explain.
    
</div>

**Double-click this cell to type your answer here:** 

<div class="alert alert-block alert-warning">

**1.7** - If this model were curved, rather than linear, do you believe the $R^2$ could be higher? Explain.
    
</div>

**Double-click this cell to type your answer here:** 

### 2.0 - Polynomial regression

Recall that simple linear regression follows this formula:

$$
\hat{y} = \beta_{0} + \beta_{1}x
$$
Where:

- $\beta_0$ is the intercept

- $\beta_1$ is the slope (coefficient of $x$)

- $\hat{y}$ is the predicted `default_rate`

- $x$ is the value of `SAT_avg`

If we want to capture the curvature in a scatter plot by creating a non-linear model, we can use a technique called **polynomial regression**. For example, we could use a degree 2 polynomial (quadratic), which looks like this:

$$
\hat{y} = \beta_{0} + \beta_{1}x + \beta_{2}x^2
$$

Where:

- $\beta_0$ is the intercept

- $\beta_1$ is the coefficient of $x$ (linear term)

- $\beta_2$ is the coefficient of $x^2$ (squared term)

- $\hat{y}$ is the predicted `default_rate`

- $x$ is the `SAT_avg`

Below, we visualize the fit of this degree-2 polynomial (quadratic) model between `SAT_avg` and `default_rate`:

In [ ]:
## Run this code but do not edit it
# create scatterplot: default_rate ~ SAT_avg, with degree 2 polynomial model overlayed
gf_point(default_rate ~ SAT_avg, data = sample_dat) %>% gf_lm(formula = y ~ poly(x, 2), color = "orange")

<div class="alert alert-block alert-warning">

**2.1** - Make a prediction: Will this polynomial regression model have a higher or lower $R^2$ value than the linear regression model? Justify your reasoning.
    
</div>

**Double-click this cell to type your answer here:** 

Let's test your prediction. To do so, we'll first need to fit the polynomial model. We can fit a degree 2 polynomial to the data using the `poly()` function inside of the `lm()` function. Run the cell below to see how it's done.

In [ ]:
## Run this code but do not edit it
# degree 2 polynomial model for default_rate ~ SAT_avg
sat_model_2 <- lm(default_rate ~ poly(SAT_avg, 2), data = sample_dat)
sat_model_2

The equation for this model would be

$$
\hat{y}=4.065-8.391x+4.355x^2
$$

Where:

- $\beta_0=4.065$ is the intercept

- $\beta_1=-8.391$ is the coefficient of $x$, the linear term

- $\beta_2=4.355$ is the coefficient of $x^2$, the squared term

- $\hat{y}$ is the predicted `default_rate`

- $x$ is the `SAT_avg`

<div class="alert alert-block alert-warning">

**2.2 -** Use the `summary` command on `sat_model_2` to see summary information about the quadratic model.
    
</div>

In [ ]:
# Your code goes here


<div class="alert alert-block alert-info">

**Check yourself:** The $R^2$ value shown in the model summary should be 0.5802 </div>

<div class="alert alert-block alert-warning">

**2.3 -** How does this model's $R^2$ value compare to that of the linear model? Was your prediction right? Explain.
    
</div>

**Double-click this cell to type your answer here:** 

This analysis raises a natural question: Why stop at degree 2? By raising the degree, we can add more curves to our model, potentially better fitting the data! Let's visualize what happens when we increase the degree in our polynomial regression models.

#### Degree 3 Polynomial Model

$$
\hat{y} = \beta_{0} + \beta_{1}x + \beta_{2}x^2 + \beta_{3}x^3
$$

In [ ]:
## Run this code but do not edit it
# create scatterplot: default_rate ~ SAT_avg, with degree 3 polynomial model overlayed
gf_point(default_rate ~ SAT_avg, data = sample_dat) %>% gf_lm(formula = y ~poly(x, 3), color = "orange") + ylim(-4,12)

#### Degree 5 Polynomial Model

$$
\hat{y} = \beta_{0} + \beta_{1}x + \beta_{2}x^2 + \beta_{3}x^3 + \beta_{4}x^4 + + \beta_{5}x^5
$$

In [ ]:
## Run this code but do not edit it
# create scatterplot: default_rate ~ SAT_avg, with degree 5 polynomial model overlayed
gf_point(default_rate ~ SAT_avg, data = sample_dat) %>% gf_lm(formula = y ~poly(x, 5), color = "orange") + ylim(-4,12)

#### Degree 12 Polynomial Model

$$
\hat{y} = \beta_{0} + \beta_{1}x + \beta_{2}x^2 + \beta_{3}x^3 + \beta_{4}x^4 + + \beta_{5}x^5 + + \beta_{6}x^6 + ... + \beta_{12}x^{12}
$$

**Note:** The following code is pre-run, to save computer space.

In [ ]:
## Note: This code was pre-run, to save computer space
# create scatterplot: default_rate ~ SAT_avg, with degree 12 polynomial model overlayed
# gf_point(default_rate ~ SAT_avg, data = sample_dat) %>% gf_smooth(method = "lm", formula = y ~poly(x,12), color = "orange") + ylim(-4,14)

<img src="https://skewthescript.org/s/high_poly.png" width = 72% align = left>

<div class="alert alert-block alert-warning">

**2.4 -** Examine each plot for the polynmial models with degrees 3, 5, 12. Which model do you think would have the largest $R^2$ value? Why?
    
</div>

**Double-click this cell to type your answer here:** 

To determine which polynomial model fits the data the best, we will fit models for each degree (3, 5, 12).

In [ ]:
## Run this code but do not edit it
# degree 3, 5, and 12 polynomial models for default_rate ~ SAT_avg
sat_model_3 <- lm(default_rate ~ poly(SAT_avg, 3), data = sample_dat)
sat_model_5 <- lm(default_rate ~ poly(SAT_avg, 5), data = sample_dat)
sat_model_12 <- lm(default_rate ~ poly(SAT_avg, 12), data = sample_dat)

Now we can compare each model's $R^2$ value. Normally, we use the `summary` command and read the $R^2$ value. However, since we've fit so many models, we don't want to print out the entire summary for each one. 

Instead, we'll use commands like this: `summary(sat_model_1)$r.squared`. The `$` operator is used to extract just the `r.squared` element from the full `summary`. We execute this command for each model, then print the results for ease of comparison.

In [ ]:
## Run this code but do not edit it
# r-squared value for each model
r2_sat_model_1 <- summary(sat_model_1)$r.squared
r2_sat_model_2 <- summary(sat_model_2)$r.squared
r2_sat_model_3 <- summary(sat_model_3)$r.squared
r2_sat_model_5 <- summary(sat_model_5)$r.squared
r2_sat_model_12 <- summary(sat_model_12)$r.squared

# print each model's r-squared value
print(paste("The R squared value for the degree 1 model is", r2_sat_model_1))
print(paste("The R squared value for the degree 2 model is", r2_sat_model_2))
print(paste("The R squared value for the degree 3 model is", r2_sat_model_3))
print(paste("The R squared value for the degree 5 model is", r2_sat_model_5))
print(paste("The R squared value for the degree 12 model is", r2_sat_model_12))

<div class="alert alert-block alert-info">

**Check yourself:** The $R^2$ for the degree 5 model should be about 0.647 </div>

<div class="alert alert-block alert-warning">

**2.5 -** The degree 12 model has the highest $R^2$ value. Does that mean it's the "best" model? Why or why not?

**Hint:** Think about which model would do the best for predicting *the rest of the data* from the original full dataset.
    
</div>

**Double-click this cell to type your answer here:** 

### 3.0 - Prediction, model tuning, & machine learning

In prior notebooks, we've used our models to make inferences about default rates. However, sometimes in data science, we care more about predictions than we do about inferences. In particular, many data science tasks ask for making accurate predictions on *new* data - data that hadn't yet been collected when we first fit the model. This process of building models to predict new data, especially when it's automated, is called **machine learning.** 

The key to machine learning is building models that make accurate predictions on **test** data - unseen data that weren't used when fitting the model. Let's see how this works. First, let's create a test dataset of 10 randomly sampled colleges. Importantly, these are colleges that **our models didn't see while fitting**:

In [ ]:
## Run this code but do not edit it
# create a data set to test the model with 10 new, randomnly selected observations
# not used to train the model
set.seed(23)
test_dat <- sample(dat, size = 10)

<div class="alert alert-block alert-warning">

**3.1 -** Use the `head` command on the `test_dat` data set.
    
</div>

In [ ]:
# Your code goes here


The following code visualizes the new test data alongside the training data (the data we used to originally fit our models).

In [ ]:
## Run this code but do not edit it
# label train and test sets
sample_dat$phase <- "train"
test_dat$phase <- "test"

# concatenate two datasets
full_dat <- rbind(sample_dat, test_dat)

# create scatterplot: default_rate ~ SAT_avg, with degree 5 polynomial model overlayed
gf_point(default_rate ~ SAT_avg, data = full_dat, color = ~phase, shape = ~phase)

<div class="alert alert-block alert-warning">

**3.2 -** Of all the polynomial models we fit before, which do you think would do best in predicting the default rates in the test dataset? 

**Note:** Use your gut and intution here. No calculations required.
    
</div>

**Double-click this cell to type your answer here:** 

Let's see how good one of our models is at predicting default rates. The R code in the next cell uses the `predict` function to make predictions on the test dataset. In this case, output shows the predicted default rates for the 10 test set colleges, as predicted by our degree 5 model.

In [ ]:
## Run this code but do not edit it
# get predictions for degree 5 model
pred_deg5 <- predict(sat_model_5, newdata = data.frame(SAT_avg = test_dat$SAT_avg))
pred_deg5

So, how can we interpret these values? Well, the last college in our test set is `University of New Orleans`, which has an `SAT_avg` value of `1088` and a `default_rate` of `6.6`. It's shown here on the graph, alongside our degree 5 model. 

**Note:** The following code is pre-run.

In [ ]:
### Run this code but do not edit it
## create scatterplot: default_rate ~ SAT_avg, with degree 5 polynomial model overlayed
#gf_point(default_rate ~ SAT_avg, data = sample_dat, color = ~phase, shape = ~phase) %>% gf_lm(formula = y ~poly(x, 5), color = "orange") %>% gf_point(default_rate ~ SAT_avg, data = full_dat, color = ~phase, shape = ~phase) + ylim(0,19)

<img src="https://skewthescript.org/s/new_orleans.png" width = 72%, align = left>

Our degree 5 model's predicted default rate for this first data point was `4.28`.  That means that our degree 5 model under-estimates the actual value for default rate by...

$$
6.6-4.28 = 2.32
$$

The model's prediction and error is visualized in the plot below:

<img src="https://skewthescript.org/s/new_orleans_residual.png" width = 72% align = left>


So, its predicted default rate is "off" by about 2 percentage points! This is pretty amazing, considering the model had only 20 training data values and the `University of New Orleans` wasn't included among them. This is the power of machine learning! Predicting previously *unseen* data!

This is just one prediction. We're really interested in how this model performed across all its predictions. For that, let's measure its $R^2$ (prediciton strength) on the test set!

We can use the `cor` function to correlate the predictions with the actual default rates ($r$) and then square that value to get $R^2$, which gets us the prediction strength!

In [ ]:
## Run this code but do not edit it
# Get correlation between predicted and actual default rates in test set
cor(test_dat$default_rate, pred_deg5) ^ 2

We can now repeat this same process for all polynomial degrees.

In [ ]:
## Run this code but do not edit it
# Storing test set predictions for all models
pred_deg1 <- predict(sat_model_1, newdata = data.frame(SAT_avg = test_dat$SAT_avg))
pred_deg2 <- predict(sat_model_2, newdata = data.frame(SAT_avg = test_dat$SAT_avg))
pred_deg3 <- predict(sat_model_3, newdata = data.frame(SAT_avg = test_dat$SAT_avg))
pred_deg5 <- predict(sat_model_5, newdata = data.frame(SAT_avg = test_dat$SAT_avg))
pred_deg12 <- predict(sat_model_12, newdata = data.frame(SAT_avg = test_dat$SAT_avg))

# print each model's r-squared value
print(paste("The test R squared value for the degree 1 model is", cor(test_dat$default_rate, pred_deg1) ^ 2))
print(paste("The test R squared value for the degree 2 model is", cor(test_dat$default_rate, pred_deg2) ^ 2))
print(paste("The test R squared value for the degree 3 model is", cor(test_dat$default_rate, pred_deg3) ^ 2))
print(paste("The test R squared value for the degree 5 model is", cor(test_dat$default_rate, pred_deg5) ^ 2))
print(paste("The test R squared value for the degree 12 model is", cor(test_dat$default_rate, pred_deg12) ^ 2))

<div class="alert alert-block alert-info">

**Check yourself:** The $R^2$ for the degree 5 model should be about 0.6165 </div>

<div class="alert alert-block alert-warning">

**3.3 -** Compare the $R^2$ estimates for each model. Which models did well? Which models did poorly? Why do you think this is?
    
</div>

**Double-click this cell to type your answer here:** 

<div class="alert alert-block alert-warning">

**3.4 -** In machine learning, the central goal is to build our models so as to avoid "underfitting" and "overfitting" our models to the training data. What do you think these terms mean? Which of our models were underfit? Which do you think were overfit? Explain.
    
</div>

**Double-click this cell to type your answer here:** 

Recall that we built our polynomial models here with just one predictor: $x$ (`SAT_avg`). Yet, those models could end up being quite complex...

$$
\hat{y} = \beta_{0} + \beta_{1}x + \beta_{2}x^2 + \beta_{3}x^3
$$

Now, imagine that we wanted to bring in multiple predictors ($x_1$ = `SAT_avg`, $x_2$ = `net_tuition`, $x_3$ = `grad_rate`) for muliple regression. Plus, imagine that we decided to add in some polynomial terms for each of these predictors. We could end up with a model that looks ever more complicated, with literally hundreds of terms...

$$
\hat{y} = \beta_{0} + \beta_{1}x_1 + \beta_{2}x_1^2 + \beta_{3}x_1^3 + \beta_{4}x_2 + \beta_{5}x_2^2 + \beta_{5}x_2^3 + \beta_{6}x_3 + \beta_{7}x_3^2 + ...
$$

<div class="alert alert-block alert-warning">

**3.5 -** Is it always good to add more predictors and add more polynomial terms to your model? Explain why or why not.
    
</div>

**Double-click this cell to type your answer here:** 

### 4.0 - In-class prediction competition

Now you have all the tools you need to build very powerful prediction models! This means that it's time for a friendly competition :)

The code below takes the full dataset and splits it into larger train and test datasets. 80% of the colleges will go into the train dataset. 20% will go into the test dataset. Because we all are setting the same seed (`2024`), everyone will get the exact same train and test sets:

In [ ]:
## Run but do not edit this code

# set training data to be 80% of all colleges
train_size <- floor(0.8 * nrow(dat))

## sample row indeces
set.seed(2024)
train_ind <- sample(seq_len(nrow(dat)), size = train_size)

train <- dat[train_ind, ]
test <- dat[-train_ind, ]

In [ ]:
dim(train)

In [ ]:
dim(test)

Now it's time to compete! 

**Goal:** Create the most accurate prediction model of colleges' default rates.

**Evaluation:** Whichever student has the highest $R^2$ on the test set wins.

**Guidelines** Save your best model as an object called `my_model`. You are only allowed to fit models on the train set (not on the test set). You may use as many predictors and as many polynomial terms as you'd like. Just be warned: Don't fall into the trap of overfitting! Choose only the most important variables and keep your models simple, so that you can generalize well to the test set. Periodically test your model on the test set and then make adjustments as necessary. 

Go!

In [ ]:
## Your code goes here
# Replace 'grad_rate + poly(SAT_avg,3)' with your own combo of variables and poly terms
my_model <- lm(default_rate ~ grad_rate + poly(SAT_avg,3), data = train)

In [ ]:
# run this code to get the R^2 value on the test set from your model
test_predictions = predict(my_model, newdata = test)
print(paste("The test R^2 value was: ", cor(test$default_rate, test_predictions) ^ 2))

### 5.0 - NATIONWIDE prediction competition

**Competition:** We're hosting a *nationwide* competition to see which student can build the best model for predicting student loan default rates at different colleges. Here's an <a href = "https://medium.com/data-science-4-everyone/after-the-ap-data-science-champions-announced-fd405bcdb883">article</a> about last year's winners.

**Evaluation**: Across the country, all students are using the same train and test sets as you did in the prior exercise to fit and evaluate their models. Your goal: Build a model that gives the best predictions on this test set. The student models that produce the highest $R^2$ value on the test set will be announced as champions!

**Submission Process (due June 7, 2024 at 11:59pm CT)**:
 1. Print and have a parent/guardian sign the <a href = "https://drive.google.com/file/d/1JPOiYNJLtUM3QZQKlU0qhN0nSN3AZfJL/view?usp=sharing">media release form</a>. This form gives permission to feature you and publish your results, in the event that you're a finalist! Take a picture or scan the signed form and submit it during as a part of Step #2 (below).
 2. Submit <a href = "https://forms.gle/3CwV3KqPZLjsP1dJ7">this google form</a> (note: you'll have to log into a google account), which allows you to upload your media release form, model, and notebook. This counts as your final submission.

**Notes to avoid disqualification:**
 - Do not change the seed (`2024`) in the code block that splits the data into the train and test sets. Using the common seed of `2024` will ensure everyone across the country has the exact same train/test split. 
 - Make sure your model is fit using the `train` data. In other words, it should look like: `my_model <- lm(default_rate ~ ..., data = train)`.
 - Make sure your find the $R^2$ value on the `test` data, using the provided code.
 - There are ways to "cheat" on this competition by looking directly at the test set data values and designing your model to predict those values exactly (or approximately). However, based on the design of your model (which we'll see when you share your notebook), it's pretty easy for us to tell if you've done this. So, don't do it! Your submission will be discarded.

### Feedback (Required)

Please take 2 minutes to fill out <a href="https://forms.gle/ePwTHdSeAc8FvVjg7">this anonymous notebook feedback form</a>, so we can continue improving this notebook for future years!